In [1]:
# test classification dataset
from sklearn.datasets import make_classification
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, roc_auc_score, recall_score
import RIfunctions as ri
from imblearn.under_sampling import NearMiss
from sklearn.model_selection import GridSearchCV
import joblib

In [2]:
data = pd.read_csv('RI_accept.csv', sep = ',', index_col=0)

C:\Users\joshy\AppData\Roaming\Python\Python38\site-packages\numpy\lib\arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
data['issue_d'] = pd.to_datetime(data['issue_d'])

In [4]:
data = data.loc[data['issue_d'] < '2016-1-1']

In [5]:
data = pd.get_dummies(data,prefix=["purp"], columns=['purpose'], drop_first=False)

In [6]:
train_df = data.loc[data['issue_d'] < data['issue_d'].quantile(0.80)]
test_df = data.loc[data['issue_d'] >= data['issue_d'].quantile(0.80)]

In [7]:
train_df['emp_length'].fillna(train_df['emp_length'].mean(), inplace = True)
test_df['emp_length'].fillna(test_df['emp_length'].mean(), inplace=True)

c:\Users\joshy\anaconda3\lib\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [8]:
scaler = StandardScaler(copy=False)
numerical_cols = ['loan_amnt', 'emp_length', 'dti']

train_df[numerical_cols] = scaler.fit_transform(train_df[numerical_cols], train_df['charged_off'])
test_df[numerical_cols] = scaler.transform(test_df[numerical_cols])

print(scaler.mean_)

<ipython-input-8-65ee4bc3d695>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[numerical_cols] = scaler.fit_transform(train_df[numerical_cols], train_df['charged_off'])
c:\Users\joshy\anaconda3\lib\site-packages\pandas\core\indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
<ipython-input-8-65ee4bc3d695>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,co

[1.27632717e+04 1.24275388e+00 2.27269315e+01]


In [9]:
# # Downsampling. No logical strategy.

# print('train and test shape     ', train_df.shape, test_df.shape)

# train_df = ri.balanceData(train_df)
# test_df =  ri.balanceData(test_df)

# print('train and test shape     ', train_df.shape, test_df.shape)
# print('value counts train     ', train_df['charged_off'].value_counts())
# print('value counts test     ', test_df['charged_off'].value_counts())

In [10]:
all_cols = list(data.columns)

for i in all_cols:
    if i.startswith("purp"):
        numerical_cols.append(i)

print(numerical_cols) 

train_df_y = train_df['charged_off']
train_df_x = train_df[numerical_cols]

test_df_x = test_df[numerical_cols]
test_df_y = test_df['charged_off']

['loan_amnt', 'emp_length', 'dti', 'purp_Business', 'purp_car', 'purp_credit_card', 'purp_debt_consolidation', 'purp_home_improvement', 'purp_house', 'purp_major_purchase', 'purp_medical', 'purp_moving', 'purp_other', 'purp_small_business', 'purp_vacation', 'purp_wedding']


In [11]:
# ns = NearMiss(sampling_strategy='not minority')

# train_df_x, train_df_y  = ns.fit_resample(train_df_x, train_df_y)
# print(pd.DataFrame(train_df_y.value_counts()))

# test_df_x, test_df_y = ns.fit_resample(test_df_x, test_df_y)
# print(pd.DataFrame(test_df_y.value_counts()))

In [12]:
# # Synthesise binary classification dataset
# X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_redundant=5, random_state=3)
# # summarize the dataset
print(train_df_x.shape, train_df_y.shape)

(4918780, 16) (4918780,)


In [13]:
model = joblib.load("./random_forest.joblib")
print(model.get_params())

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 100, 'max_features': 'log2', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'verbose': 0, 'warm_start': False}


In [14]:
model_2 = BalancedRandomForestClassifier(**model.get_params())
model_2.set_params(max_depth=12)
print(model_2.get_params())
model_2.fit(train_df_x, train_df_y)

{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'entropy', 'max_depth': 12, 'max_features': 'log2', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 100, 'n_jobs': None, 'oob_score': False, 'random_state': 42, 'replacement': False, 'sampling_strategy': 'auto', 'verbose': 0, 'warm_start': False}


BalancedRandomForestClassifier(criterion='entropy', max_depth=12,
                               max_features='log2', random_state=42)

In [15]:
# Hyperparameter tuning. Took over 11 hours to run.

# params_rf ={
#           'n_estimators': [100,300,400,500, 600,1000],
#           'criterion': ["gini", "entropy"],
#           'max_depth': [100,200, 300, 400, 1000],
#           'max_features': ["log2", "sqrt"],
#           'bootstrap':[True, False]
# }

# grid_rf = GridSearchCV(estimator=model, param_grid=params_rf ,cv = 3, scoring="neg_mean_squared_error", verbose = 1, n_jobs=-1)
# grid_rf.fit(train_df_x, train_df_y)

# best_hyperparams = grid_rf.best_params_
# print("Best hyperparameters: \n", best_hyperparams )
# best_rf = grid_rf.best_estimator_

In [16]:
# Saving the model

# joblib.dump(best_rf, "./random_forest.joblib")

In [17]:
# 10 fold cross validation
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
# cv_scores_train = cross_val_score(estimator= model, X=train_df_x, y=train_df_y, scoring='roc_auc', cv=cv, n_jobs=-1)
# print("(Training Set) The cross-validation accuracy score for optimised Random Forest model after a 10 fold cross validation: %.3f (%.3f)" % (mean(cv_scores_train), std(cv_scores_train)))

# cv_scores_test = cross_val_score(estimator= model, X=test_df_x, y=test_df_y, scoring='roc_auc', cv=cv, n_jobs=-1)
# print("(Test Set) The cross-validation accuracy score for optimised Random Forest model after a 10 fold cross validation: %.3f (%.3f)" % (mean(cv_scores_test), std(cv_scores_test)))

In [18]:
# 10 fold cross roc_auc
cv_scores_train = cross_val_score(estimator= model_2, X=train_df_x, y=train_df_y, scoring='roc_auc', cv=10, n_jobs=-1)
print("(Training Set) The cross-validation AUC for optimised Random Forest model after a 10 fold cross validation: %.3f (%.3f)" % (mean(cv_scores_train), std(cv_scores_train)))

cv_scores_test = cross_val_score(estimator= model_2, X=test_df_x, y=test_df_y, scoring='roc_auc', cv=10, n_jobs=-1)
print("(Test Set) The cross-validation AUC for optimised Random Forest model after a 10 fold cross validation: %.3f (%.3f)" % (mean(cv_scores_test), std(cv_scores_test)))

: 

In [ ]:
yhat = model.predict(test_df_x)
print(roc_auc_score(test_df_y, yhat), recall_score(test_df_y, yhat, pos_label=1), recall_score(test_df_y, yhat, pos_label=0))

c:\Users\joshy\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- purp_Business
Feature names seen at fit time, yet now missing:
- purp_educational
- purp_renewable_energy

  warnings.warn(message, FutureWarning)


ValueError: X has 16 features, but RandomForestClassifier is expecting 17 features as input.